### 1. Import the essential libraries

In [1]:
import os
import sys
import cv2
import random
import joblib
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

from skimage.color import rgb2lab, lab2rgb
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float

from tqdm import tqdm_notebook

### 1.1 Define some helpful functions

In [2]:
def surf_feature_extraction(image, hessianThreshold=400, nOctaves=3, n_keypoints=10):
    # construct a mask for the segment
    surf = cv2.xfeatures2d.SURF_create(hessianThreshold=hessianThreshold, nOctaves=nOctaves)
    key_points = surf.detect(image)
    key_points = sorted(key_points, key=lambda x: -x.response)[:n_keypoints]
    key_points, description = surf.compute(image, key_points)
    
    if len(key_points) != 0:
        description = description.flatten()
    else:
        description = np.array([])
        
    fearute_vector_size = (n_keypoints * 64)
    if description.size < fearute_vector_size:
        description = np.concatenate([description, np.zeros(fearute_vector_size - description.size)])
    
    return (key_points, description)

In [3]:
def calc_energy_and_amplitude(filtered_img):
    '''
    Return a list of values:
      * Local Energy = summing up the squared value of each matrix value from a response matrix
      * Mean Amplitude = sum of absolute values of each matrix value from a response matrix
    '''
    local_energy = np.sum(np.square(filtered_img))
    mean_amplitude = np.sum(np.absolute(filtered_img))
    return [local_energy, mean_amplitude]

def gabor_feature_extraction(image_lab_L, theta_range=[0, np.pi/6, np.pi/4, np.pi/3, np.pi/2, 2*np.pi/3, 3*np.pi/4, 5*np.pi/6], scale_range=[3, 6, 13, 28, 58]):
    features = []
    for scale in scale_range:
        for angle in theta_range:
            g_kernel = cv2.getGaborKernel(ksize=(20, 20), sigma=scale, theta=angle, lambd=7, gamma=0.9, psi=1.5, ktype=cv2.CV_32F)
            filtered_img = cv2.filter2D(image_lab_L, cv2.CV_8UC3, g_kernel)
            features.extend(calc_energy_and_amplitude(filtered_img))
    return features

### 2. Set the paths for *train* and *test* data

In [4]:
train_images_path = os.path.join("/media/andretri7/WD/Datasets/flower_images/flower_images")
res_path          = os.path.join('.', 'res_all')

### 2.1. Get the Train Images & the Color Palette

In [5]:
train_images = joblib.load(os.path.join(res_path,'trainImages.joblib'))
colorPalette = joblib.load(os.path.join(res_path,'colorPalette.joblib'))
print (f'{type(train_images)}\n{type(colorPalette)}')

<class 'numpy.ndarray'>
<class 'sklearn.cluster.k_means_.MiniBatchKMeans'>


In [6]:
np.sort(train_images)

array(['0022.png', '0024.png', '0046.png', '0052.png', '0070.png',
       '0082.png', '0088.png', '0090.png', '0103.png', '0112.png',
       '0123.png', '0134.png', '0140.png', '0142.png', '0148.png',
       '0152.png', '0178.png', '0191.png', '0200.png'], dtype='<U8')

### 3. Get Images and Perform the Preprocessing (cont.)
  * #### 3.1: Read Image
  * #### 3.2: Normalize Values from [0, 255] to [0, 1]
  * #### 3.3: Convert Image from RGB to LAB
  * #### 3.4: Get the Superpixels for an Image using the SLIC Segmentation Algorithm
  * #### 3.5: Extract the SURF Features from an Image per Superpixel
  * #### 3.6: Extract the Gabor Features from an Image per Superpixel
  * #### 3.7: Get the Dominant Color per Superpixel
  * #### 3.8: Associate each feature vector {SURF, Gabor} with the corresponding color class of the dominant color in the color palette

In [7]:
dataset = None

for img in tqdm_notebook(train_images):
    # 3.1: Read Image
    trainImage = cv2.imread(os.path.join(train_images_path, img))
    trainImage = cv2.cvtColor(trainImage, cv2.COLOR_BGR2RGB)
    # 3.2: Normalize Values from [0, 255] to [0, 1]
    trainImageScaled = trainImage/255
    # 3.3: Convert Image from RGB to LAB
    trainImageLAB = rgb2lab(trainImageScaled)
    # 3.4: Get the Superpixels for an Image using the SLIC Segmentation Algorithm
    segments = slic(trainImageLAB, convert2lab=False, n_segments=600, sigma=5) 
    # 3.5: Extract the SURF Features from an Image per Superpixel
    # 3.6: Extract the Gabor Features from an Image per Superpixel
    # 3.7: Get the Dominant Color per Superpixel
    for (i, segment) in enumerate(np.unique(segments)):
        # Construct a mask for the segment
        mask = np.zeros(trainImage.shape[:2], dtype = "uint8")
        mask[segments == segment] = 255
        superpixel = cv2.bitwise_and(trainImageLAB, trainImageLAB, mask=mask)
        
        superpixel_colors = pd.DataFrame(np.vstack(superpixel[:, :, 1:3]), columns=['a', 'b'])    
        
        maximal_color = KMeans(n_clusters=5, n_init=4, max_iter=100, n_jobs=-1)
        maximal_color.fit(superpixel_colors.values)
        superpixel_colors['y_color'] = maximal_color.labels_
        superpixel_dominant_color = superpixel_colors.loc[superpixel_colors['y_color'] == superpixel_colors['y_color']  
                                                     .value_counts().index[1]].apply(lambda x: x.median())[['a', 'b']]  

        superpixel_dominant_color_class = colorPalette.predict(superpixel_dominant_color.values.reshape(1,-1))
          
        surf_vector = surf_feature_extraction(np.uint8(superpixel[:,:,0]), n_keypoints=5)[1]
        gabor_vector = np.array(gabor_feature_extraction(np.uint8(superpixel[:,:,0])))
        color_vector = np.array(superpixel_dominant_color_class[0])
        feature_vector = np.hstack((surf_vector, gabor_vector, color_vector))
        
        if (dataset is None):
            dataset = np.array(feature_vector)
        else:
            dataset = np.row_stack((dataset, feature_vector))

/home/andretri7/anaconda3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:156: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  segments_z = grid_z[slices]
/home/andretri7/anaconda3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:157: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  segments_y = grid_y[slices]
/home/andretri7/anaconda3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:158: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[

/home/andretri7/anaconda3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:156: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  segments_z = grid_z[slices]
/home/andretri7/anaconda3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:157: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  segments_y = grid_y[slices]
/home/andretri7/anaconda3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:158: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[

/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklear

/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklear

/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklear

/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklear

/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklear

/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:156: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  segments_z = grid_z[slices]
/home/andretri7/anaconda3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:157: FutureWarning: Using a non-t

/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklear

/home/andretri7/anaconda3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:156: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  segments_z = grid_z[slices]
/home/andretri7/anaconda3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:157: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  segments_y = grid_y[slices]
/home/andretri7/anaconda3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:158: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[

In [11]:
pdDataset = pd.DataFrame(dataset)
pdDataset

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,400
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,181.0,1956.0,316.0,394.0,172.0,200.0,108.0,460.0,166.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5612.0,12166.0,7690.0,14526.0,4824.0,13731.0,5649.0,11745.0,3251.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1368.0,7767.0,1173.0,11232.0,1372.0,6685.0,967.0,9086.0,1088.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,892.0,1335.0,293.0,8004.0,1100.0,10780.0,1340.0,12802.0,1502.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,559.0,2832.0,422.0,2823.0,559.0,3146.0,620.0,6414.0,924.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15054.0,15384.0,15678.0,18451.0,14867.0,17601.0,11775.0,11044.0,5068.0,9.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,22106.0,15293.0,13629.0,20214.0,17494.0,20890.0,18386.0,17386.0,19962.0,5.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17415.0,15381.0,9203.0,20350.0,18308.0,19446.0,17480.0,20223.0,14685.0,5.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13121.0,16866.0,11658.0,18973.0,14787.0,20950.0,13652.0,15806.0,22432.0,6.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,816.0,1956.0,360.0,4397.0,779.0,3078.0,696.0,4776.0,840.0,0.0


# CHECKPOINT
* ### Save the train image dataset for the following notebooks

In [12]:
if not os.path.exists(os.path.join('.', 'res_all')):
    os.makedirs(os.path.join('.', 'res_all'))
    
save_path = os.path.join('.', 'res_all')

In [13]:
pdDataset.to_pickle(os.path.join(save_path, 'pdDataset.pkl'))